In [1]:
from ipywidgets import interact, interactive, fixed, interact_manual
from mpl_toolkits.mplot3d import axes3d
import ipywidgets as widgets
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
import math
%matplotlib notebook

A = [-200, -100, -170, 15]
a = [-1, -1, -6.5, 0.7]
b = [0, 0, 11, 0.6]
c = [-10, -10, -6.5, 0.7]
xbar1 = [1, 0, -0.5, -1]
xbar2 = [0, 0.5, 1.5, 1]
xmin = -1.5
xmax = 1.5
ymin = -0.5
ymax = 2
stepRate = math.pow(10, -10)
stepSize = (abs(xmin) + abs(xmax)) * stepRate

In [2]:
def Vx(x1, x2, A, a, b, C, xbar1, xbar2):
    return np.sum([
        A[i] * np.exp((a[i] * (x1 - xbar1[i])**2) + (b[i] * (x1 - xbar1[i])) *
                      (x2 - xbar2[i]) + C[i] * (x2 - xbar2[i])**2)
        for i in range(len(A))
    ])


def move_new(x1, x2, mut_size, max_mut=1):

    x1_lst, x2_lst, new_score = [], [], []
    x_tups = np.array([(1, 0), (1, -1), (0, -1), (-1, -1), (-1, 0), (-1, 1),
                       (0, 1), (1, 1)])

    x_tup = get_coors(x1, x2, mut_size)

    new_x = (x_tups * x_tup) + (x1, x2)
    new_score = [Vx(x[0], x[1], A, a, b, c, xbar1, xbar2) for x in new_x]
#     print("XY")
    print([x for x in new_x])
#     print("V")
    print([Vx(x[0], x[1], A, a, b, c, xbar1, xbar2) for x in new_x])
#     print(f"{new_score.index(min(new_score)), new_x[new_score.index(min(new_score))], min(new_score)}\n")
    return new_score.index(min(new_score)), new_x[new_score.index(
        min(new_score))], min(new_score)


def get_coors(x1, x2, mut_size):

    new_x1 = abs(np.random.normal(0, mut_size, 1)[0])
    new_x2 = abs(np.random.normal(0, mut_size, 1)[0])
    qiv = 1.96 * mut_size

    if -qiv < new_x1 < qiv and -qiv < new_x2 < qiv:
        return new_x1, new_x2
    return get_coors(x1, x2, mut_size)


def getScheme_data(iters, plot="No"):
    linspace = np.linspace(0, 9, iters)
    answer = []

    for x in linspace:
        answer.append(0.5 +
                      math.erf((math.sqrt(math.pi) / 2) * (x - 2)) * -0.5)
    return answer


def mp_hillclimber(n_it, n_p):
    
    xcoords = np.linspace(-0.85, -0.35, n_p)
    ycoords = np.linspace(1.35, 0.75, n_p)
    coords = [(x,y) for x,y in zip(xcoords, ycoords)]
    cur_score = np.mean([Vx(x[0], x[1], A, a, b, c, xbar1, xbar2) for x in coords])
    cur_coords = coords
    for i in range(n_it):
        new_coors, new_score = [], []
        for x1, x2 in cur_coords:
            new = move_new(x1, x2, mut_size = 0.05)
            new_coors.append(new[1])
            new_score.append(new[2])

        if np.mean(new_score) <= cur_score:
            cur_score = np.mean(new_score)
            cur_coords = new_coors
        print(cur_score)

def mp_hillclimber(n_it, n_p, climb_scheme="Hillclimb"):
    if climb_scheme == "simAn": scheme = getScheme_data(n_it)
    xcoords = [-0.774170186599428] # np.linspace(-0.85, 0.5, n_p)
    ycoords = [1.22439901501604] # np.linspace(1.35, 0.0, n_p) theta (x0) = 1.37210046981424
# theta1 (y0) = -1.53124691795276
    coords = [(x, y) for x, y in zip(xcoords, ycoords)]
    cur_coords = coords
    cur_scores = [Vx(x[0], x[1], A, a, b, c, xbar1, xbar2) for x in coords]
    cur_score = np.mean(cur_scores)
    coors_list = {"x": [xcoords], "y": [ycoords], "z": [cur_scores]}
    for i in range(n_it):
        new_coors, new_score = [], []
        print(cur_coords)
        for x1, x2 in cur_coords:
            print(f"######## cur_coords: {(cur_coords, i)}")
#         for x1, x2 in cur_coords:
            print(f"********* {x1, x2} **********\n")
            new = move_new(x1, x2, mut_size=0.05)
            new_coors.append(new[1])
            new_score.append(new[2])
            print(f"********* curscore, newscore {cur_score, np.mean(new_score)} **********\n")
        if np.mean(new_score) < cur_score:
            print("FUCK YEAH MOTHAFAKA")
            cur_score = np.mean(new_score)
            cur_coords = new_coors
            print(cur_coords)
            coors_list["x"].append(list(np.array(new_coors)[:, 0]))
            coors_list["y"].append(list(np.array(new_coors)[:, 1]))
            coors_list["z"].append(new_score)
        elif climb_scheme == "simAn":
            if scheme[i] > np.random.uniform():
                cur_score = np.mean(new_score)
                cur_coords = new_coors
                coors_list["x"].append(list(np.array(new_coors)[:, 0]))
                coors_list["y"].append(list(np.array(new_coors)[:, 1]))
                coors_list["z"].append(new_score)
                
        print("\n NEXT ITER \n")
    return coors_list


data = mp_hillclimber(100, 6)
print(len(data), len(data["x"]), len(data["x"][0]))

[(-0.774170186599428, 1.22439901501604)]
######## cur_coords: ([(-0.774170186599428, 1.22439901501604)], 0)
********* (-0.774170186599428, 1.22439901501604) **********

[array([-0.72808099,  1.22439902]), array([-0.72808099,  1.17812347]), array([-0.77417019,  1.17812347]), array([-0.82025938,  1.17812347]), array([-0.82025938,  1.22439902]), array([-0.82025938,  1.27067456]), array([-0.77417019,  1.27067456]), array([-0.72808099,  1.27067456])]
[-131.0681491459469, -122.59154129238954, -124.67193792707745, -122.84778777790841, -124.64018103928154, -122.60129241168735, -131.1355206111008, -135.8895441013596]
********* curscore, newscore (-129.8430418086566, -135.8895441013596) **********

FUCK YEAH MOTHAFAKA
[array([-0.72808099,  1.27067456])]

 NEXT ITER 

[array([-0.72808099,  1.27067456])]
######## cur_coords: ([array([-0.72808099,  1.27067456])], 1)
********* (-0.7280809908287905, 1.2706745608039738) **********

[array([-0.69306409,  1.27067456]), array([-0.69306409,  1.26990444]),

In [3]:
X = np.array(np.linspace(xmin, xmax, 200))
Y = np.array(np.linspace(ymin, ymax, 200))
Z = []

for x1 in X:
    temp = []
    for y1 in Y:
        V = Vx(x1, y1, A, a, b, c, xbar1, xbar2)
        if V > 100:
            temp.append(0)
        else:
            temp.append(V)
    Z.append(temp)
Z = np.array(Z)

In [4]:
def showPlot(i=0):
    print(X.shape, Y.shape, Z.shape)
    contours = plt.contour(X, Y, Z, 3, colors='black')
    plt.clabel(contours, inline=True, fontsize=8)

    plt.imshow(
        Z,
        extent=[np.min(X), np.max(X),
                np.min(Y), np.max(Y)],
        origin='lower',
        cmap='RdGy',
        alpha=0.7)
    plt.colorbar()
    for j in range(len(data["x"][i])):
        plt.scatter(data["x"][i][j], data["y"][i][j], c="b")
    plt.show

y = interactive(showPlot, i=(0, len(data["x"]) - 1, 1))
display(y)

# theta (x0) = -0.774170186599428
# theta1 (y0) = 1.22439901501604

interactive(children=(IntSlider(value=0, description='i', max=23), Output()), _dom_classes=('widget-interact',…

In [12]:
from sympy import *
A = [-200, -100, -170, 15]
a = [-1, -1, -6.5, 0.7]
b = [0, 0, 11, 0.6]
c = [-10, -10, -6.5, 0.7]
xbar1 = [1, 0, -0.5, -1]
xbar2 = [0, 0.5, 1.5, 1]
xmin = -1.5
xmax = 1.5
ymin = -0.5
ymax = 2
stepRate = math.pow(10, -10)
stepSize = (abs(xmin) + abs(xmax)) * stepRate

x1 = Symbol("x1")
x2 = Symbol("x2")
f = (-200 * exp(
    (-1 * (x1 - 1)**2) + (0 * (x1 - 0)) *
    (x2 - 0) + -10 * (x2 - 0)**2)) + (-100 * exp(
        (-1 * (x1 - 0)**2) + (0 * (x1 - 0)) *
        (x2 - 0.5) + -10 * (x2 - 0.5)**2)) + (-170 * exp(
            (-6.5 * (x1 - -0.5)**2) + (11 * (x1 - -0.5)) *
            (x2 - 1.5) + -6.5 * (x2 - 1.5)**2)) + (15 * exp(
                (0.7 * (x1 - -1)**2) + (11 * (x1 - -1)) *
                (x2 - 1) + 0.7 * (x2 - 1)**2))

fpx1 = f.diff(x1)
fpx2 = f.diff(x2)
print(fpx1)
print(fpx2)
grad = [fpx1, fpx2]

# Data
theta = -0.85  #x
theta1 = 1.35 #y
alpha = .0001
iterations = 0
check = 0
precision = 1/1000000
printData = True
maxIterations = 1000

while True:
    temptheta = theta - alpha*N(fpx1.subs(x1,theta).subs(x2,theta1)).evalf()
    temptheta1 = theta1 - alpha*N(fpx2.subs(x2,theta1)).subs(x1,theta).evalf()

    #If the number of iterations goes up too much, maybe theta (and/or theta1)
    #is diverging! Let's stop the loop and try to understand.
    iterations += 1
    if iterations > maxIterations:
        print("Too many iterations. Adjust alpha and make sure that the function is convex!")
        printData = False
        break

    #If the value of theta changes less of a certain amount, our goal is met.
    if abs(temptheta-theta) < precision and abs(temptheta1-theta1) < precision:
        break

    #Simultaneous update
    theta = temptheta
    theta1 = temptheta1

if printData:
    print("The function "+str(f)+" converges to a minimum")
    print("Number of iterations:",iterations,sep=" ")
    print("theta (x0) =",temptheta,sep=" ")
    print("theta1 (y0) =",temptheta1,sep=" ")

200*x1*exp(-x1**2 - 10*(x2 - 0.5)**2) - 200*(-2*x1 + 2)*exp(-10*x2**2 - (x1 - 1)**2) - 170*(-13.0*x1 + 11*x2 - 23.0)*exp(-6.5*(x1 + 0.5)**2 + (11*x1 + 5.5)*(x2 - 1.5) - 6.5*(x2 - 1.5)**2) + 15*(1.4*x1 + 11*x2 - 9.6)*exp(0.7*(x1 + 1)**2 + (11*x1 + 11)*(x2 - 1) + 0.7*(x2 - 1)**2)
4000*x2*exp(-10*x2**2 - (x1 - 1)**2) - 100*(-20*x2 + 10.0)*exp(-x1**2 - 10*(x2 - 0.5)**2) - 170*(11*x1 - 13.0*x2 + 25.0)*exp(-6.5*(x1 + 0.5)**2 + (11*x1 + 5.5)*(x2 - 1.5) - 6.5*(x2 - 1.5)**2) + 15*(11*x1 + 1.4*x2 + 9.6)*exp(0.7*(x1 + 1)**2 + (11*x1 + 11)*(x2 - 1) + 0.7*(x2 - 1)**2)
The function -100*exp(-x1**2 - 10*(x2 - 0.5)**2) - 200*exp(-10*x2**2 - (x1 - 1)**2) - 170*exp(-6.5*(x1 + 0.5)**2 + (11*x1 + 5.5)*(x2 - 1.5) - 6.5*(x2 - 1.5)**2) + 15*exp(0.7*(x1 + 1)**2 + (11*x1 + 11)*(x2 - 1) + 0.7*(x2 - 1)**2) converges to a minimum
Number of iterations: 76
theta (x0) = -0.774170186599428
theta1 (y0) = 1.22439901501604


In [6]:
from sympy import *

x = Symbol('x')
y = Symbol('y')
z = Symbol('z')


f = x**2 + y**2 - 2*x*y

# First partial derivative with respect to x
fpx = f.diff(x)

# First partial derivative with respect to y
fpy = f.diff(y)

# Gradient
grad = [fpx,fpy]

# Data
theta = 830  #x
theta1 = 220 #y
alpha = .01
iterations = 0
check = 0
precision = 1/1000000
printData = True
maxIterations = 1000

while True:
    temptheta = theta - alpha*N(fpx.subs(x,theta).subs(y,theta1)).evalf()
    temptheta1 = theta1 - alpha*N(fpy.subs(y,theta1)).subs(x,theta).evalf()

    #If the number of iterations goes up too much, maybe theta (and/or theta1)
    #is diverging! Let's stop the loop and try to understand.
    iterations += 1
    if iterations > maxIterations:
        print("Too many iterations. Adjust alpha and make sure that the function is convex!")
        printData = False
        break

    #If the value of theta changes less of a certain amount, our goal is met.
    if abs(temptheta-theta) < precision and abs(temptheta1-theta1) < precision:
        break

    #Simultaneous update
    theta = temptheta
    theta1 = temptheta1

if printData:
    print("The function "+str(f)+" converges to a minimum")
    print("Number of iterations:",iterations,sep=" ")
    print("theta (x0) =",temptheta,sep=" ")
    print("theta1 (y0) =",temptheta1,sep=" ")
    
    
# Output
#
# The function x**2 - 2*x*y + y**2 converges to a minimum
# Number of iterations: 401
# theta (x0) = 525.000023717248
# theta1 (y0) = 524.999976282752


The function x**2 - 2*x*y + y**2 converges to a minimum
Number of iterations: 401
theta (x0) = 525.000023717248
theta1 (y0) = 524.999976282752


In [7]:
cur_x = 3 # The algorithm starts at x=3
rate = 0.01 # Learning rate
precision = 0.000001 #This tells us when to stop the algorithm
previous_step_size = 1 #
max_iters = 10000 # maximum number of iterations
iters = 0 #iteration counter
df = lambda x: 2*(x+5) #Gradient of our function

In [8]:
while previous_step_size > precision and iters < max_iters:
    prev_x = cur_x #Store current x value in prev_x
    cur_x = cur_x - rate * df(prev_x) #Grad descent
    previous_step_size = abs(cur_x - prev_x) #Change in x
    iters = iters+1 #iteration count
    print("Iteration",iters,"\nX value is",cur_x) #Print iterations
    
print("The local minimum occurs at", cur_x)

Iteration 1 
X value is 2.84
Iteration 2 
X value is 2.6832
Iteration 3 
X value is 2.529536
Iteration 4 
X value is 2.37894528
Iteration 5 
X value is 2.2313663744
Iteration 6 
X value is 2.0867390469119997
Iteration 7 
X value is 1.9450042659737599
Iteration 8 
X value is 1.8061041806542846
Iteration 9 
X value is 1.669982097041199
Iteration 10 
X value is 1.5365824551003748
Iteration 11 
X value is 1.4058508059983674
Iteration 12 
X value is 1.2777337898784
Iteration 13 
X value is 1.152179114080832
Iteration 14 
X value is 1.0291355317992152
Iteration 15 
X value is 0.9085528211632309
Iteration 16 
X value is 0.7903817647399662
Iteration 17 
X value is 0.6745741294451669
Iteration 18 
X value is 0.5610826468562635
Iteration 19 
X value is 0.44986099391913825
Iteration 20 
X value is 0.3408637740407555
Iteration 21 
X value is 0.23404649855994042
Iteration 22 
X value is 0.1293655685887416
Iteration 23 
X value is 0.026778257216966764
Iteration 24 
X value is -0.07375730792737258
It

X value is -4.998172272697408
Iteration 416 
X value is -4.9982088272434595
Iteration 417 
X value is -4.998244650698591
Iteration 418 
X value is -4.998279757684619
Iteration 419 
X value is -4.998314162530927
Iteration 420 
X value is -4.998347879280309
Iteration 421 
X value is -4.998380921694703
Iteration 422 
X value is -4.998413303260809
Iteration 423 
X value is -4.998445037195593
Iteration 424 
X value is -4.998476136451681
Iteration 425 
X value is -4.998506613722648
Iteration 426 
X value is -4.998536481448195
Iteration 427 
X value is -4.998565751819231
Iteration 428 
X value is -4.998594436782846
Iteration 429 
X value is -4.998622548047189
Iteration 430 
X value is -4.998650097086245
Iteration 431 
X value is -4.9986770951445205
Iteration 432 
X value is -4.99870355324163
Iteration 433 
X value is -4.998729482176797
Iteration 434 
X value is -4.998754892533261
Iteration 435 
X value is -4.998779794682596
Iteration 436 
X value is -4.998804198788944
Iteration 437 
X value i

In [9]:
Vx(-0.85, 1.35, A, a, b, c, xbar1, xbar2)

-100.91503524596283

In [10]:
# %matplotlib notebook
# i = 15
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# ax.plot_wireframe(X, Y, Z, rstride=10, cstride=10)
# for j in range(len(data["x"])):
#     ax.scatter(data["x"][j][0], 
#                 data["y"][j][0], 
#                 data["z"][j][0],#(len(data["z"][0])-1)-j], 
#                 c="red")
# plt.show()

In [11]:
quadratic steepest descent hessian

SyntaxError: invalid syntax (<ipython-input-11-d819da6e7683>, line 1)